# Global Imports

In [ ]:
import os
import pprint
import random as rnd
import sys
from pathlib import Path as Pt
from time import sleep
from typing import Dict, Iterator, List, Optional, Set, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import pyvista as pv
from matplotlib import gridspec
from numpy import random as nrnd

from utils_002 import (Template, gen_xyz, load_cfg, prep_cuboid,
                       read_large_file_lines)

In [ ]:
Ax = 5.85E-10
Ay = 1.78E-10
Az = 4.41E-10

# A1x = Ax / (2 ** (1 / 3))
# A1y = Ay / (2 ** (1 / 3))
# A1z = Az / (2 ** (1 / 3))

# A1x = Ax / (7 ** (1 / 3))
# A1y = Ay / (7 ** (1 / 3))
# A1z = Az / (7 ** (1 / 3))

A1x = Ax / (14 ** (1 / 3))
A1y = Ay / (14 ** (1 / 3))
A1z = Az / (14 ** (1 / 3))

g100 = 0.41
g010 = 0.54
g001 = 0.22

Ω = A1x * A1y * A1z

EX = 2 * g100 * A1y * A1z
EY = 2 * g010 * A1z * A1x
EZ = 2 * g001 * A1x * A1y

Eisol = EX + EY + EZ
E3 = EY + EX
E2 = EZ + EY
E1 = EZ + EX

kB = 1.380649e-23
T = 300.0
kT = kB * T
Ceq = np.exp(- Eisol / kT)

print(
    """
Sizes:
    Ax: {:.5e},    Ay: {:.5e},    Az: {:.5e}
   A1x: {:.5e},   A1y: {:.5e},   A1z: {:.5e}

Volume:
   Ω = A1x * A1y * A1z = {:.5e} * {:.5e} * {:.5e} = {:.5e}

Surface tensions:
   g100: {:.5e}, g010: {:.5e}, g001: {:.5e}

Energys:
1)
   EX = 2.0 * g100 * A1y * A1z = 2.0 * {:.5e} * {:.5e} * {:.5e} = {:.5e}
   EY = 2.0 * g010 * A1z * A1x = 2.0 * {:.5e} * {:.5e} * {:.5e} = {:.5e}
   EZ = 2.0 * g001 * A1x * A1y = 2.0 * {:.5e} * {:.5e} * {:.5e} = {:.5e}

2)
   Eisol = EX + EY + EZ = {:.5e} + {:.5e} + {:.5e} = {:.5e}

3)
   E3 = EY + EX = {:.5e} + {:.5e} = {:.5e}
   E2 = EZ + EY = {:.5e} + {:.5e} = {:.5e}
   E1 = EZ + EX = {:.5e} + {:.5e} = {:.5e}

kB = {:.5e}; T = {:.5f};
kT = kB * T = {:.5e} * {:.5f} = {:.5e};
Ceq = exp(- Eisol / kT ) = exp(- {:.5e} / {:.5e} ) = {:.5e};
""".format(
        Ax, Ay, Az,
        A1x, A1y, A1z,
        A1x, A1y, A1z,
        Ω,
        g100, g010, g001,
        g100, A1y, A1z, EX,
        g010, A1z, A1x, EY,
        g001, A1x, A1y, EZ,
        EX, EY, EZ, Eisol,
        EY, EX, E3,
        EZ, EY, E2,
        EZ, EX, E1,
        kB, T,
        kB, T, kT,
        Eisol, kT, Ceq,
    ).strip()
)

In [ ]:
def process_directory(dir_path: Pt, axis_mode: str = "X") -> Dict:
    """
    Читает конфиг, историю и состояния, вычисляет среднюю скорость роста вдоль axis_mode.
    """
    dir_path = Pt(dir_path)
    cfg_file = dir_path / "InitSettings.ini"
    states_file = dir_path / "TimeStates.txt"
    history_file = dir_path / "sim_history.txt"
    if not (cfg_file.exists() and states_file.exists() and history_file.exists()):
        raise FileNotFoundError(f"Missing files in {dir_path}")

    cfg = load_cfg(path_to_config=cfg_file)
    cfg["id"] = dir_path.name.split("_", 1)[0]
    sx, sy, sz = cfg["Sx"], cfg["Sy"], cfg["Sz"]

    # Читаем только последний ряд истории (mk_step_history)
    lines = history_file.read_text(encoding="utf-8").splitlines()
    mk_steps = np.fromstring(lines[-1], dtype=np.float32, sep=":").astype(np.int32)

    # Собираем размеры кластеров построчно, лениво:
    cluster_sizes = [line.count("1") for line in read_large_file_lines(states_file)]

    # разность размеров и шагов
    ds = np.diff(cluster_sizes)
    dt = np.diff(mk_steps)

    # нормируем по сечению оси
    area = {"X": sy * sz, "Y": sx * sz, "Z": sx * sy}[axis_mode]
    cfg["mean_velocity"] = (ds / dt / area).mean()

    return cfg


axes_dirs = {
    "X": Pt(  #
        r"AxesVelocity\AxisX"
        #
    ),
    "Y": Pt(  #
        r"AxesVelocity\AxisY"
        #
    ),
    "Z": Pt(  #
        r"AxesVelocity\AxisZ"
        #
    ),
}

results = {}
for axis, base in axes_dirs.items():
    data = []
    for root, _, files in os.walk(base):
        if {"InitSettings.ini", "TimeStates.txt"}.issubset(files):
            cfg = process_directory(Pt(root), axis_mode=axis)
            data.append((cfg["dg"], cfg["mean_velocity"]))

    results[axis] = np.array(data).T

In [ ]:
# 1. Глобальные настройки стиля
plt.rcParams.update(
    {
        "font.family": "serif",
        "font.serif": ["Times New Roman"],
        "font.size": 8,
        "axes.labelsize": 8,
        "axes.titlesize": 8,
        "legend.fontsize": 7,
        "xtick.labelsize": 7,
        "ytick.labelsize": 7,
        "lines.linewidth": 0.5,
        "lines.markersize": 2.0,
        "xtick.direction": "in",
        "ytick.direction": "in",
        "axes.grid": True,
        "grid.linestyle": ":",
        "grid.linewidth": 0.4,
    }
)

# 2. Создаём фигуру под журнал (~8.9 cm)
fig, ax = plt.subplots(figsize=(3.5, 2), dpi=600)


datasets = [
    ("", -Eisol, "-", 0.50, ".0"),
    ("±Eisol", Eisol, "-", 0.50, ".0"),
    ("", -EX, (0, (5, 2)), 0.50, ".50"),
    ("±EX", EX, (0, (5, 2)), 0.50, ".50"),
    ("", -EY, (0, (5, 2)), 0.50, ".25"),
    ("±EY", EY, (0, (5, 2)), 0.50, ".25"),
    ("", -EZ, (0, (5, 2)), 0.50, ".75"),
    ("±EZ", EZ, (0, (5, 2)), 0.50, ".75"),
    ("", -E3, (0, (5, 2, 1, 2)), 0.50, ".75"),
    ("±E3=EY+EX", E3, (0, (5, 2, 1, 2)), 0.50, ".75"),
    ("", -E2, (0, (5, 2, 1, 2)), 0.50, ".50"),
    ("±E2=EZ+EY", E2, (0, (5, 2, 1, 2)), 0.50, ".50"),
    ("", -E1, (0, (5, 2, 1, 2)), 0.50, ".25"),
    ("±E1=EZ+EX", E1, (0, (5, 2, 1, 2)), 0.50, ".25"),
]

for label, x, ls, lw, color in datasets:
    ax.axvline(
        x=x, ymin=-1, ymax=1, linewidth=lw, color=color, linestyle=ls, label=label
    )


# 3. Данные и стили
datasets = [
    ("Axis X", results["X"][0], results["X"][1], "s", "-", 0.65, ".50"),
    ("Axis Y", results["Y"][0], results["Y"][1], "o", "-", 0.65, ".25"),
    ("Axis Z", results["Z"][0], results["Z"][1], "^", "-", 0.65, ".75"),
]

for label, x, y, marker, ls, lw, color in datasets:
    ax.plot(
        x,
        y,
        marker=marker,
        linewidth=lw,
        linestyle=ls,
        color=color,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=5,
        label=label,
    )

# 4. Оформление осей, сетки, легенды
ax.set_xlabel("Value of $dg$")
ax.set_ylabel("Mean velocity of the\ncrystal/liquid interface")

ax.grid(True, which="both", linestyle=":", linewidth=0.4)
ax.tick_params(length=2, width=0.5)

# ax.legend(bbox_to_anchor=(1.0, 0.0), loc="best", frameon=False)
# ax.legend(bbox_to_anchor=(1.0, 0.0), loc='lower center')
ax.legend(bbox_to_anchor=(1.0, 1), loc="upper left", borderaxespad=0.0, frameon=False)

plt.tight_layout()

plt.savefig(
    Pt(  #
        r"RES\AxesVelocity"
        #
    )
    / "Axes_W_markers.png",
    format="png",
)

plt.show()
plt.close()